## 2. LIMPIEZA

### 1. Importamos las librerías necesarias.

In [44]:
# Herramientas para EDA
import pandas as pd
import numpy as np
import sidetable as stb
from datetime import timedelta
from datetime import date, datetime

# Herramientas para la Visualización
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors 
from matplotlib.gridspec import GridSpec
import plotly as py
import plotly.graph_objs as go

### 2. Abrimos el archivo con el que vamos a trabajar.

In [45]:
df = pd.read_csv('../archivos/bikes.csv', index_col=0)
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,spring,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,spring,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,spring,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


### 3. Después de un EDA inicial, procedemos a la limpieza de nuestros datos.

3.1. Eliminación de columnas.

In [46]:
# Eliminamos la columna instant, porqué es redundante

df.drop(['instant'], axis=1, inplace=True)

3.2. Traducción de columnas y valores.

In [47]:
# Renombramos las columnas para que esté toda nuestra info en un mismo idioma
nuevas_columnas= ['fecha', 'estacion', 'año', 'mes', 'dias_festivos', 'dias_de_la_semana', 'dias_laborales', 'clima', 'temperatura', 'sensacion_termica', 'humedad', 'velocidad_viento', 'clientes_casuales', 'clientes_registrados', 'total_bicis']
df.columns = nuevas_columnas

3.3. Corregimos los días de la semana.

In [48]:
for i,v in enumerate(df['fecha']):
    days= ['1', '2', '3', '4', '5', '6', '7']
    d= datetime.strptime(v, '%d-%m-%Y') #damos la vuelta a la fecha
    df.loc[i,'dia_semana'] = days[d.weekday()] 

In [49]:
df.head()

,fecha,estacion,año,mes,dias_festivos,dias_de_la_semana,dias_laborales,clima,temperatura,sensacion_termica,humedad,velocidad_viento,clientes_casuales,clientes_registrados,total_bicis,dia_semana
0,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,1
1,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801,2
2,03-01-2018,spring,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349,3
3,04-01-2018,spring,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562,4
4,05-01-2018,spring,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600,5


In [50]:
df.drop(['dias_de_la_semana'], axis=1, inplace=True)

3.4. Las estaciones estaban intercambiadas, ponemos los nombres correctos.

In [51]:
df['estacion'].unique()

array(['spring', 'summer', 'autumn', 'winter'], dtype=object)

In [52]:
dic_est = {'spring' : 'invierno', 'winter' : 'otoño', 'summer' : 'primavera', 'autumn' : 'verano'}
df['estacion'] = df['estacion'].map(dic_est)

In [53]:
df.sample(5)

,fecha,estacion,año,mes,dias_festivos,dias_laborales,clima,temperatura,sensacion_termica,humedad,velocidad_viento,clientes_casuales,clientes_registrados,total_bicis,dia_semana
617,10-09-2019,verano,1,9,0,1,1,23.916653,28.28270,50.3750,17.333771,1118,6407,7525,2
689,21-11-2019,otoño,1,11,0,1,1,14.486653,18.24460,61.3750,6.917482,615,4531,5146,4
89,31-03-2018,primavera,0,3,0,1,3,11.001653,12.87875,91.8333,14.582282,179,1506,1685,6
149,30-05-2018,primavera,0,5,1,0,1,30.066653,33.55460,68.5000,8.792075,1549,2549,4098,3
134,15-05-2018,primavera,0,5,0,0,2,23.062500,26.80000,86.7083,10.249593,1582,2971,4553,2


3.5. Corregimos los días festivos.

New Years Day January  1

Martin Luther King’s Birthday January 16

Lincolns Birthday February 13

President’s Day February 20

Cesar Chavez’ Birthday March 31

Memorial Day May 29

Independence Day July 4

Bloody Thursday July 5

Harry Bridges’ Birthday July 28

Labor Day September 4

Veterans’ Day November 10

Thanksgiving Day November 23

Day After Thanksgiving November 24

Christmas Eve December 24

Christmas Day December 25

New Year’s Eve December 31

3.3. Estandarizamos los outliers.

In [56]:
# Ahora nos centramos en nuestros outliers
numericas = df.select_dtypes(include=np.number) 
numericas.head()

,año,mes,dias_festivos,dias_laborales,clima,temperatura,sensacion_termica,humedad,velocidad_viento,clientes_casuales,clientes_registrados,total_bicis
0,0,1,0,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,0,1,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,0,1,0,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,0,1,0,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,0,1,0,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [57]:
lista_columnas = numericas.columns  

In [58]:
def detectar_outliers(lista_columnas, dataframe): 
    
    dicc_indices = {} 
    
    for col in lista_columnas:
        
        Q1 = np.nanpercentile(numericas[col], 25)
        Q3 = np.nanpercentile(numericas[col], 75)
     
        IQR = Q3 - Q1
        
        outlier_step = 1.5 * IQR
    
        outliers_data = dataframe[(dataframe[col] < Q1 - outlier_step) | (dataframe[col] > Q3 + outlier_step)]
        
        
        if outliers_data.shape[0] > 0: 
        
            dicc_indices[col] = (list(outliers_data.index)) 
        
    return dicc_indices 

In [59]:
indices= detectar_outliers(lista_columnas, df)  

In [60]:
indices.keys() 

dict_keys(['dias_festivos', 'humedad', 'velocidad_viento', 'clientes_casuales'])

In [61]:
outliers_humedad = list(indices['humedad'])
outliers_velocidad_viento = list(indices['velocidad_viento'])
outliers_festivos = list(indices['dias_festivos'])
outliers_clientes = list(indices['clientes_casuales'])

In [62]:
#generamos un for loop para reemplazar los valores atípicos de 'humedad':
for i in outliers_humedad:
    df.replace(numericas.loc[i,'humedad'], numericas['humedad'].median(), inplace=True)

In [63]:
#generamos un for loop para reemplazar los valores atípicos de 'velocidad_viento':
for i in outliers_velocidad_viento:
    df.replace(numericas.loc[i,'velocidad_viento'], numericas['velocidad_viento'].median(), inplace=True)

In [64]:
#generamos un for loop para reemplazar los valores atípicos de 'dias festivos':
for i in outliers_festivos:
    df.replace(numericas.loc[i,'dias_festivos'], numericas['dias_festivos'].median(), inplace=True)

In [65]:
#generamos un for loop para reemplazar los valores atípicos de 'clientes_casuales':
for i in outliers_clientes:
    df.replace(numericas.loc[i,'clientes_casuales'], numericas['clientes_casuales'].median(), inplace=True)